## 3. Data Preparation

This script merges all the currency files and also merge the daily level crypto prices and market prices (dow jones indicator).

Furthermore, custom variables are created such as polarity, Bullish Ratio and Negative Share.

**Output is the daily level summarised data required for modeling exercise**

In [2]:
import pandas as pd
import numpy as np
import string
# !pip install --user -U nltk # if you don't have it
import nltk 
#nltk.download() # if you don't have it
from nltk.corpus import stopwords
import nltk.tokenize 
from nltk.stem import WordNetLemmatizer

#word count
from collections import Counter

#word clouds
import heapq
#from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import collections
import matplotlib.cm as cm
from matplotlib import rcParams
from tqdm import tqdm

#sentiment analysis
from textblob import TextBlob

#text preprocessing
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')
from scipy import stats

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Cryptocurrency Prices

Daily level crypto prices are downloaded from internet across different exchanges and average prices are calculated to get final bitcoin prices 

In [13]:
Bitstamp_ETHUSD_1h = pd.read_csv('Bitstamp_ETHUSD_1h.csv', skiprows=1)
gemini_ETHUSD_1hr = pd.read_csv('gemini_ETHUSD_1hr.csv', skiprows=1)
gemini_ETHUSD_1hr.head()


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1629936000000,2021-08-26 00:00:00,ETHUSD,3229.84,3235.77,3227.31,3230.00,94.089697
1,1629932400000,2021-08-25 23:00:00,ETHUSD,3217.17,3238.87,3209.76,3229.84,614.891020
2,1629928800000,2021-08-25 22:00:00,ETHUSD,3224.31,3240.01,3211.76,3217.17,705.052088
3,1629925200000,2021-08-25 21:00:00,ETHUSD,3210.10,3225.18,3207.03,3224.31,309.627907
4,1629921600000,2021-08-25 20:00:00,ETHUSD,3216.76,3239.96,3200.99,3210.10,609.511790


In [14]:

Bitstamp_ETHUSD_1h.rename(columns={'close':'Close_Bitstamp', 'Volume ETH':'Volume_Bitstamp', 'open':'Open_Bitstamp',
                                 'high':'High_Bitstamp','low':'Low_Bitstamp'},inplace=True)

gemini_ETHUSD_1hr.rename(columns={'Close':'Close_gemini', 'Volume':'Volume_gemini','Open':'Open_gemini',
                                 'High':'High_gemini','Low':'Low_gemini'},inplace=True)

In [15]:
Bitstamp_ETHUSD_1h.head()

,unix,date,symbol,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp,Volume USD
0,1629936000,2021-08-26 00:00:00,ETH/USD,3226.98,3235.07,3209.05,3212.70,82.824241,2.660894e+05
1,1629932400,2021-08-25 23:00:00,ETH/USD,3218.55,3239.26,3209.52,3230.85,188.617285,6.093942e+05
2,1629928800,2021-08-25 22:00:00,ETH/USD,3228.18,3236.93,3212.31,3217.87,353.647196,1.137991e+06
3,1629925200,2021-08-25 21:00:00,ETH/USD,3208.71,3225.12,3204.86,3222.86,270.213107,8.708590e+05
4,1629921600,2021-08-25 20:00:00,ETH/USD,3218.30,3241.16,3200.00,3209.36,364.267443,1.169065e+06


In [16]:
#BTC_prices  = gemini_BTCUSD_day[['Date','Close_gemini','Volume_gemini']].merge(Bitfinex_BTCUSD_1h[['date','Close_Bitfinex','Volume_Bitfinex']], left_on ='Date',right_on='date', how='left')
gemini_ETHUSD_1hr['Date'] = pd.to_datetime(gemini_ETHUSD_1hr['Date'])
Bitstamp_ETHUSD_1h['Date'] = pd.to_datetime(Bitstamp_ETHUSD_1h['date'])

BTC_prices  = gemini_ETHUSD_1hr[['Date','Open_gemini','High_gemini','Low_gemini','Close_gemini','Volume_gemini']].merge(Bitstamp_ETHUSD_1h[['Date','Open_Bitstamp','High_Bitstamp','Low_Bitstamp','Close_Bitstamp','Volume_Bitstamp']], on='Date', how='left')

BTC_prices.head()

,Date,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp
0,2021-08-26 00:00:00,3229.84,3235.77,3227.31,3230.00,94.089697,3226.98,3235.07,3209.05,3212.70,82.824241
1,2021-08-25 23:00:00,3217.17,3238.87,3209.76,3229.84,614.891020,3218.55,3239.26,3209.52,3230.85,188.617285
2,2021-08-25 22:00:00,3224.31,3240.01,3211.76,3217.17,705.052088,3228.18,3236.93,3212.31,3217.87,353.647196
3,2021-08-25 21:00:00,3210.10,3225.18,3207.03,3224.31,309.627907,3208.71,3225.12,3204.86,3222.86,270.213107
4,2021-08-25 20:00:00,3216.76,3239.96,3200.99,3210.10,609.511790,3218.30,3241.16,3200.00,3209.36,364.267443


In [17]:
BTC_prices['Volume_BTC_gemini'] = BTC_prices['Volume_gemini']*BTC_prices['Close_gemini']

BTC_prices['Volume_BTC_Bitstamp'] = BTC_prices['Volume_Bitstamp']*BTC_prices['Close_Bitstamp']

BTC_prices['Close_prices'] = (BTC_prices['Close_gemini']+BTC_prices['Close_Bitstamp'])/2
BTC_prices['Open_prices'] = (BTC_prices['Open_gemini']+BTC_prices['Open_Bitstamp'])/2
BTC_prices['High_prices'] = (BTC_prices['High_gemini']+BTC_prices['High_Bitstamp'])/2
BTC_prices['Low_prices'] = (BTC_prices['Low_gemini']+BTC_prices['Low_Bitstamp'])/2
BTC_prices['Volume_BTC'] = BTC_prices['Volume_BTC_gemini']+BTC_prices['Volume_BTC_Bitstamp']
BTC_prices.head()

,Date,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp,Volume_BTC_gemini,Volume_BTC_Bitstamp,Close_prices,Open_prices,High_prices,Low_prices,Volume_BTC
0,2021-08-26 00:00:00,3229.84,3235.77,3227.31,3230.00,94.089697,3226.98,3235.07,3209.05,3212.70,82.824241,3.039097e+05,2.660894e+05,3221.350,3228.410,3235.420,3218.180,5.699992e+05
1,2021-08-25 23:00:00,3217.17,3238.87,3209.76,3229.84,614.891020,3218.55,3239.26,3209.52,3230.85,188.617285,1.986000e+06,6.093942e+05,3230.345,3217.860,3239.065,3209.640,2.595394e+06
2,2021-08-25 22:00:00,3224.31,3240.01,3211.76,3217.17,705.052088,3228.18,3236.93,3212.31,3217.87,353.647196,2.268272e+06,1.137991e+06,3217.520,3226.245,3238.470,3212.035,3.406263e+06
3,2021-08-25 21:00:00,3210.10,3225.18,3207.03,3224.31,309.627907,3208.71,3225.12,3204.86,3222.86,270.213107,9.983364e+05,8.708590e+05,3223.585,3209.405,3225.150,3205.945,1.869195e+06
4,2021-08-25 20:00:00,3216.76,3239.96,3200.99,3210.10,609.511790,3218.30,3241.16,3200.00,3209.36,364.267443,1.956594e+06,1.169065e+06,3209.730,3217.530,3240.560,3200.495,3.125659e+06


### Merge different datasets

Custom variable creating and summary data at HOURLY level is created

In [18]:
def data_manipulation(df):

    df['length_tweet'] = df['tweet_new2'].apply(len)
    df['Date'] = pd.to_datetime(df['created_date'])
    summarised = df.groupby(['Date'], as_index=False).agg({'neg':'mean','neu':'mean','pos':'mean','compound':'mean','retweet_count':'mean','like_count':'mean','author id':'count',
                                                                                  "unique_id": pd.Series.nunique, 'length_tweet':'mean','followers_count':'mean'})
    
    data_sum = df.groupby(['Date','sentiment'], as_index=False)['unique_id'].count()
    data_sum1 = data_sum.pivot(index='Date', columns='sentiment', values='unique_id')
    data_sum1['Total_hr_tweets'] = data_sum1['negative']+data_sum1['positive']+data_sum1['neutral']
    data_sum1 = data_sum1.reset_index()
    
    summarised = summarised.merge(data_sum1, on='Date', how='left')
 
    summarised['Bullish_Ratio'] = summarised['positive']/summarised['negative']
    summarised['Negative_share'] = summarised['negative']/summarised['Total_hr_tweets']
    summarised = summarised.rename(columns={'neg':'avg_neg_scr','neu':'avg_neu_scr','pos':'avg_pos_scr','compound':'avg_cmp_scr','retweet_count':'avg_retweet_cnt','like_count':'avg_like_cnt','author id':'tweet_vol',
                                                                                  "unique_id": 'uniq_tweet_vol', 'length_tweet':'avg_tweet_len','followers_count':'avg_followers_cnt'})
    summarised['Date']=pd.to_datetime(summarised['Date']).dt.tz_localize(None)
    return summarised

final_data_senti1=pd.read_csv('final_data_senti_july1.csv')
summarised_1 = data_manipulation(final_data_senti1)

In [19]:
summarised_1.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-07-01 00:00:00,0.019014,0.850309,0.130299,0.322297,1369.962907,1.645935,2669,1292,127.158861,2413.022855,113,1010,1546,2669,13.681416,0.042338
1,2021-07-01 01:00:00,0.016242,0.854976,0.128778,0.308966,2216.534413,2.252812,2223,1091,127.018893,1710.888889,104,937,1182,2223,11.365385,0.046784
2,2021-07-01 02:00:00,0.014271,0.845937,0.139787,0.325600,1725.593437,1.622249,2499,1217,126.502201,2313.409764,120,1001,1378,2499,11.483333,0.048019
3,2021-07-01 03:00:00,0.011617,0.848367,0.140008,0.349539,2065.467465,0.762076,2505,1160,129.096607,1885.437525,78,962,1465,2505,18.782051,0.031138
4,2021-07-01 04:00:00,0.015265,0.837811,0.143675,0.341826,1909.296520,1.840835,2155,1148,130.377262,2985.108121,93,827,1235,2155,13.279570,0.043155


In [20]:
final_data_senti1=pd.read_csv('final_data_senti_july31.csv')
summarised_2 = data_manipulation(final_data_senti1)
summarised_2.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-07-15 00:00:00,0.006860,0.879896,0.112483,0.293279,3818.697701,0.543913,2653,998,121.612137,2787.021485,90,1111,1452,2653,16.133333,0.033924
1,2021-07-15 01:00:00,0.006033,0.881175,0.112793,0.300170,4391.608671,0.396463,3506,1259,119.513120,2133.222476,101,1332,2073,3506,20.524752,0.028808
2,2021-07-15 02:00:00,0.004383,0.882767,0.112850,0.315494,4256.229615,1.149528,4133,1208,119.237842,1898.815630,94,1444,2595,4133,27.606383,0.022744
3,2021-07-15 03:00:00,0.005484,0.883346,0.111171,0.298992,4518.388347,0.538133,3278,1127,120.914582,1501.055827,88,1244,1946,3278,22.113636,0.026846
4,2021-07-15 04:00:00,0.005264,0.887692,0.107044,0.284488,4547.158014,1.228423,3082,999,121.905905,1806.054510,64,1292,1726,3082,26.968750,0.020766


In [21]:
final_data_senti1=pd.read_csv('final_data_senti_aug15.csv')
summarised_3 = data_manipulation(final_data_senti1)
summarised_3.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-08-01 00:00:00,0.023602,0.907889,0.068509,0.124591,1065.199648,1.648197,3411,1398,112.559660,1925.152155,272,2074,1065,3411,3.915441,0.079742
1,2021-08-01 01:00:00,0.030619,0.882396,0.086990,0.136420,1335.472990,1.581994,3110,1359,113.015756,1315.472347,379,1571,1160,3110,3.060686,0.121865
2,2021-08-01 02:00:00,0.028597,0.882820,0.088586,0.166267,1482.301365,2.810641,2857,1315,117.655233,1492.045152,215,1506,1136,2857,5.283721,0.075254
3,2021-08-01 03:00:00,0.014523,0.876407,0.109073,0.237788,5346.561884,0.874623,4969,1444,112.606963,1000.242101,191,1758,3020,4969,15.811518,0.038438
4,2021-08-01 04:00:00,0.017545,0.889429,0.093030,0.175774,3876.500750,1.204799,6001,1783,107.683886,1362.519747,371,1820,3810,6001,10.269542,0.061823


In [22]:
summarised_all=summarised_1.append(summarised_2)
summarised_all=summarised_all.append(summarised_3)
summarised_all.shape

(1104, 17)

In [23]:
#summarised_all['created_date']=pd.to_datetime(summarised_all['created_date'])
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-07-01 00:00:00,0.019014,0.850309,0.130299,0.322297,1369.962907,1.645935,2669,1292,127.158861,2413.022855,113,1010,1546,2669,13.681416,0.042338
1,2021-07-01 01:00:00,0.016242,0.854976,0.128778,0.308966,2216.534413,2.252812,2223,1091,127.018893,1710.888889,104,937,1182,2223,11.365385,0.046784
2,2021-07-01 02:00:00,0.014271,0.845937,0.139787,0.325600,1725.593437,1.622249,2499,1217,126.502201,2313.409764,120,1001,1378,2499,11.483333,0.048019
3,2021-07-01 03:00:00,0.011617,0.848367,0.140008,0.349539,2065.467465,0.762076,2505,1160,129.096607,1885.437525,78,962,1465,2505,18.782051,0.031138
4,2021-07-01 04:00:00,0.015265,0.837811,0.143675,0.341826,1909.296520,1.840835,2155,1148,130.377262,2985.108121,93,827,1235,2155,13.279570,0.043155


In [24]:
summarised_all = summarised_all.merge(BTC_prices[['Date','Open_prices', 'High_prices','Low_prices','Close_prices','Volume_BTC']], left_on='Date', right_on='Date',how='left')
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC
0,2021-07-01 00:00:00,0.019014,0.850309,0.130299,0.322297,1369.962907,1.645935,2669,1292,127.158861,...,1010,1546,2669,13.681416,0.042338,2277.745,2277.745,2234.855,2246.095,3.109962e+06
1,2021-07-01 01:00:00,0.016242,0.854976,0.128778,0.308966,2216.534413,2.252812,2223,1091,127.018893,...,937,1182,2223,11.365385,0.046784,2247.170,2269.555,2245.820,2263.150,5.887730e+06
2,2021-07-01 02:00:00,0.014271,0.845937,0.139787,0.325600,1725.593437,1.622249,2499,1217,126.502201,...,1001,1378,2499,11.483333,0.048019,2263.010,2264.660,2193.080,2198.850,5.438700e+06
3,2021-07-01 03:00:00,0.011617,0.848367,0.140008,0.349539,2065.467465,0.762076,2505,1160,129.096607,...,962,1465,2505,18.782051,0.031138,2202.140,2213.740,2184.665,2185.465,2.407530e+06
4,2021-07-01 04:00:00,0.015265,0.837811,0.143675,0.341826,1909.296520,1.840835,2155,1148,130.377262,...,827,1235,2155,13.279570,0.043155,2186.675,2210.060,2181.900,2209.175,2.281976e+06


In [25]:
summarised_all['Date'] = pd.to_datetime(summarised_all['Date'])
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC
0,2021-07-01 00:00:00,0.019014,0.850309,0.130299,0.322297,1369.962907,1.645935,2669,1292,127.158861,...,1010,1546,2669,13.681416,0.042338,2277.745,2277.745,2234.855,2246.095,3.109962e+06
1,2021-07-01 01:00:00,0.016242,0.854976,0.128778,0.308966,2216.534413,2.252812,2223,1091,127.018893,...,937,1182,2223,11.365385,0.046784,2247.170,2269.555,2245.820,2263.150,5.887730e+06
2,2021-07-01 02:00:00,0.014271,0.845937,0.139787,0.325600,1725.593437,1.622249,2499,1217,126.502201,...,1001,1378,2499,11.483333,0.048019,2263.010,2264.660,2193.080,2198.850,5.438700e+06
3,2021-07-01 03:00:00,0.011617,0.848367,0.140008,0.349539,2065.467465,0.762076,2505,1160,129.096607,...,962,1465,2505,18.782051,0.031138,2202.140,2213.740,2184.665,2185.465,2.407530e+06
4,2021-07-01 04:00:00,0.015265,0.837811,0.143675,0.341826,1909.296520,1.840835,2155,1148,130.377262,...,827,1235,2155,13.279570,0.043155,2186.675,2210.060,2181.900,2209.175,2.281976e+06


In [26]:
summarised_all['polarity'] = stats.gmean(summarised_all.iloc[:, [1,3]], axis=1)

In [27]:
summarised_all.head()
summarised_all.columns

Index(['Date', 'avg_neg_scr', 'avg_neu_scr', 'avg_pos_scr', 'avg_cmp_scr',
       'avg_retweet_cnt', 'avg_like_cnt', 'tweet_vol', 'uniq_tweet_vol',
       'avg_tweet_len', 'avg_followers_cnt', 'negative', 'neutral', 'positive',
       'Total_hr_tweets', 'Bullish_Ratio', 'Negative_share', 'Open_prices',
       'High_prices', 'Low_prices', 'Close_prices', 'Volume_BTC', 'polarity'],
      dtype='object')

In [28]:
summarised_all = summarised_all[summarised_all['Date'] >= '2021-07-01'][['Date', 'avg_neg_scr', 'avg_neu_scr', 'avg_pos_scr', 'avg_cmp_scr',
       'avg_retweet_cnt', 'avg_like_cnt', 'tweet_vol', 'uniq_tweet_vol',
       'avg_tweet_len', 'avg_followers_cnt', 'negative', 'neutral', 'positive',
       'Bullish_Ratio', 'Negative_share', 'Open_prices',
       'High_prices', 'Low_prices', 'Close_prices', 'Volume_BTC', 'polarity']]
summarised_all.to_csv('model_file_final_btc.csv')

In [29]:
summarised_all['pct_change'] = summarised_all.Close_prices.pct_change()

In [30]:
def direction(df):

    if (df['pct_change'] <0):
        return 0
    elif (df['pct_change'] >=0):
        return 1

summarised_all['direction']= summarised_all.apply(direction, axis = 1)

In [31]:
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC,polarity,pct_change,direction
0,2021-07-01 00:00:00,0.019014,0.850309,0.130299,0.322297,1369.962907,1.645935,2669,1292,127.158861,...,13.681416,0.042338,2277.745,2277.745,2234.855,2246.095,3.109962e+06,0.049774,NaN,NaN
1,2021-07-01 01:00:00,0.016242,0.854976,0.128778,0.308966,2216.534413,2.252812,2223,1091,127.018893,...,11.365385,0.046784,2247.170,2269.555,2245.820,2263.150,5.887730e+06,0.045735,0.007593,1.0
2,2021-07-01 02:00:00,0.014271,0.845937,0.139787,0.325600,1725.593437,1.622249,2499,1217,126.502201,...,11.483333,0.048019,2263.010,2264.660,2193.080,2198.850,5.438700e+06,0.044663,-0.028412,0.0
3,2021-07-01 03:00:00,0.011617,0.848367,0.140008,0.349539,2065.467465,0.762076,2505,1160,129.096607,...,18.782051,0.031138,2202.140,2213.740,2184.665,2185.465,2.407530e+06,0.040330,-0.006087,0.0
4,2021-07-01 04:00:00,0.015265,0.837811,0.143675,0.341826,1909.296520,1.840835,2155,1148,130.377262,...,13.279570,0.043155,2186.675,2210.060,2181.900,2209.175,2.281976e+06,0.046832,0.010849,1.0


### dow jones prices

Daily level market prices are merged with the summarised tweet dataset

In [32]:
dow=pd.read_csv('$DOWI_Barchart_Interactive_Chart_Intraday_60m_08_21_2021.csv', skiprows=[0,642])
dow.tail()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest
635,2021-08-20 13:00:00,35132.16,35155.60,35096.65,35098.28,-36.72,18653679,NaN
636,2021-08-20 14:00:00,35098.73,35130.90,35056.21,35085.53,-12.75,20315106,NaN
637,2021-08-20 15:00:00,35085.61,35154.06,35081.67,35117.69,32.16,49183911,NaN
638,2021-08-20 16:00:00,35117.06,35120.08,35116.27,35120.08,2.39,35539860,NaN
639,2021-08-20 17:00:00,35120.08,35120.08,35120.08,35120.08,0.00,227449,NaN


In [33]:
dow['Date']=pd.to_datetime(dow['Date Time'])
dow['index_change']=dow['Close'].pct_change()
dow.head()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest,Date,index_change
0,2021-05-11 12:00:00,34298.00,34305.31,34115.15,34145.67,0.00,34485086,NaN,2021-05-11 12:00:00,NaN
1,2021-05-11 13:00:00,34145.68,34303.58,34134.82,34269.57,123.90,32900368,NaN,2021-05-11 13:00:00,0.003629
2,2021-05-11 14:00:00,34270.29,34303.65,34182.43,34274.98,5.41,33717216,NaN,2021-05-11 14:00:00,0.000158
3,2021-05-11 15:00:00,34274.78,34337.85,34154.06,34270.17,-4.81,62506021,NaN,2021-05-11 15:00:00,-0.000140
4,2021-05-11 16:00:00,34269.88,34273.77,34269.16,34269.16,-1.01,37515412,NaN,2021-05-11 16:00:00,-0.000029


In [34]:
summarised_all=summarised_all.merge(dow[['Date','Close']], on='Date', how='left')
summarised_all['Close'] = summarised_all['Close'].fillna(method='ffill')
summarised_all['Close'] = summarised_all['Close'].fillna(34502)
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC,polarity,pct_change,direction,Close
0,2021-07-01 00:00:00,0.019014,0.850309,0.130299,0.322297,1369.962907,1.645935,2669,1292,127.158861,...,0.042338,2277.745,2277.745,2234.855,2246.095,3.109962e+06,0.049774,NaN,NaN,34502.0
1,2021-07-01 01:00:00,0.016242,0.854976,0.128778,0.308966,2216.534413,2.252812,2223,1091,127.018893,...,0.046784,2247.170,2269.555,2245.820,2263.150,5.887730e+06,0.045735,0.007593,1.0,34502.0
2,2021-07-01 02:00:00,0.014271,0.845937,0.139787,0.325600,1725.593437,1.622249,2499,1217,126.502201,...,0.048019,2263.010,2264.660,2193.080,2198.850,5.438700e+06,0.044663,-0.028412,0.0,34502.0
3,2021-07-01 03:00:00,0.011617,0.848367,0.140008,0.349539,2065.467465,0.762076,2505,1160,129.096607,...,0.031138,2202.140,2213.740,2184.665,2185.465,2.407530e+06,0.040330,-0.006087,0.0,34502.0
4,2021-07-01 04:00:00,0.015265,0.837811,0.143675,0.341826,1909.296520,1.840835,2155,1148,130.377262,...,0.043155,2186.675,2210.060,2181.900,2209.175,2.281976e+06,0.046832,0.010849,1.0,34502.0


In [35]:
summarised_all.to_csv('input_model_file_new.csv')

In [36]:
summarised_all.shape

(1104, 25)